# Analysis of immunization from the CDC
For this project we'll be looking at 2017 data on immunizations from the CDC. 

## Step 1
We calculate the proportion of children in the dataset who had a mother with the education levels equal to less than high school (<12), high school (12), more than high school but not a college graduate (>12) and college degree.

In [20]:
import pandas as pd
import numpy as np
df=pd.read_csv("NISPUF17.csv",index_col=0)
    
Prop={"less than high school":len(df[df["EDUC1"]==1])*100/len(df),"high school":len(df[df["EDUC1"]==2])*100/len(df),"more than high school but not college":len(df[df["EDUC1"]==3])*100/len(df),"college":len(df[df["EDUC1"]==4])*100/len(df)}
Prop

{'less than high school': 10.202002459160372,
 'high school': 17.235201124187597,
 'more than high school but not college': 24.588090637625154,
 'college': 47.97470577902688}

## Step 2

Let's explore the relationship between being fed breastmilk as a child and getting a seasonal influenza vaccine from a healthcare provider. We calculate the average number of influenza vaccines for those children we know received breastmilk as a child and those who know did not.


In [21]:
df=df.dropna(subset=["P_NUMFLU"])
average_influenza_doses={'Average number of influenza vaccines for children we know received breastmilk':np.average(df[df["CBF_01"]==1]["P_NUMFLU"]),'Average number of influenza vaccines for children we know did not received breastmilk':np.average(df[df["CBF_01"]==2]["P_NUMFLU"])}

average_influenza_doses

{'Average number of influenza vaccines for children we know received breastmilk': 1.8799187420058687,
 'Average number of influenza vaccines for children we know did not received breastmilk': 1.5963945918878317}

## Step 3
It would be interesting to see if there is any evidence of a link between vaccine effectiveness and sex of the child. We calculate the ratio of the number of children who contracted chickenpox but were vaccinated against it (at least one varicella dose) versus those who were vaccinated but did not contract chicken pox. 

In [22]:
df=pd.read_csv("NISPUF17.csv",index_col=0)
df=df.dropna(subset=["P_NUMVRC"])
NumVacHaveMale=len(df[(df["P_NUMVRC"]>0)&(df["SEX"]==1)&(df["HAD_CPOX"]==1)])
NumVacHaveFemale=len(df[(df["P_NUMVRC"]>0)&(df["SEX"]==2)&(df["HAD_CPOX"]==1)])
number_of_male=len(df[(df["P_NUMVRC"]>0)&(df["SEX"]==1)&(df["HAD_CPOX"]==2)])
number_of_female=len(df[(df["P_NUMVRC"]>0)&(df["SEX"]==2)&(df["HAD_CPOX"]==2)])
rMale=NumVacHaveMale/number_of_male
rFemale=NumVacHaveFemale/number_of_female
chickenpox_by_sex={"male":rMale,"female":rFemale}

chickenpox_by_sex

{'male': 0.009675583380762664, 'female': 0.0077918259335489565}

## Step 4
A correlation is a statistical relationship between two variables. If we wanted to know if vaccines work, we might look at the correlation between the use of the vaccine and whether it results in prevention of the infection or disease . In this step, we see if there is a correlation between having had the chicken pox and the number of chickenpox vaccine doses given (varicella).


In [25]:
import scipy.stats as stats
import pandas as pd
df=pd.read_csv("NISPUF17.csv",index_col=0)
df=df.dropna(subset=["P_NUMVRC"])
df=df[(df["HAD_CPOX"]!=77)&(df["HAD_CPOX"]!=99)]
corr_chickenpox, pval=stats.pearsonr(df["HAD_CPOX"],df["P_NUMVRC"])
    
corr_chickenpox,pval

(0.0704487346014804, 2.7780263182866215e-18)

###### 
Some notes on interpreting the answer. If the `had_chickenpox_column` is either `1` (for yes) or `2` for no, and that the `num_chickenpox_vaccine_column` is the number of doses a child has been given of the varicella vaccine, then a positive correlation (e.g. `corr > 0`) would mean that an increase in `had_chickenpox_column` (which means more no) would mean an increase in the `num_chickenpox_vaccine_column` (which means more doses of vaccine). If `corr < 0` then there is a negative correlation, indicating that having had chickenpox is related to an increase in the number of vaccine doses. Also, `pval` refers to the probability the relationship observed is significant. In this case `pval` is very very small (will end in `e-18` indicating a very small number), which means the result unlikely to be by chance.

This is not really the full picture, since we are not looking at when the dose was given. It is possible that children had chickenpox and then their parents went to get them the vaccine.